In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#Import packages
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from itertools import count
from collections import deque

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import os

#Import Custom Classes

from Source.nn_model_dqn import QNetwork
from Source.dqn_rcv_agent import Agent, ReplayBuffer, EpsilonGreedyStrategy
from Source.env_manager import EnvManager
from Source.misc_fun.utils import plot, get_moving_average, Generate_BeamDir, All_Exhaustive_RateMeas
from Source.PER import PrioritizedReplayBuffer

In [3]:
#Hyper-parameters
BUFFER_SIZE = int(1e5)      #replay buffer size
BATCH_SIZE = 128             #minibatch size
GAMMA = 0.999                #discount factor
ALPHA = 1.0                  #non-stationarity parameter
TAU = 1e-3                  #for soft update of target parameters
LR = 5e-4                   #learning rate
TEST_EVERY = 1600            #how often to test the network
eps_start = 1.0
eps_end = 0.01
eps_decay = 0.9987 #125e-6
PER_ALPHA = 0.6
PER_BETA = 0.4
PRIORITIZED_REPLAY = False

episodes = 4200 #3100 train, 500 test
seed = 0                    #random seed number
#%%
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

inp_fptr = open("ricianch_variation.txt")
ch_randvals = inp_fptr.read().splitlines()
ch_randvals = [np.complex(a.replace('i','j')) for a in ch_randvals]

In [4]:
print(ch_randvals[0])

#print(np.complex(ch_randvals[0].replace('i','j')))

(1.1128+0.0082264j)


In [7]:
#Testing (beam_dir, beam_width) Beamset
from Source.antenna.ula import steervec
from Source.misc_fun.utils import Generate_BeamDirs, plotbeam

refinelevels = np.array([0,1,2])
beamset = Generate_BeamDirs(8,refinelevels)

print("Beamset: {}".format([np.round(x*180/np.pi, decimals=2) for x in beamset]))
print(len(beamset))
print(beamset)
tx_beam = steervec(8,beamset[-2], 0)
print("Ntx: {}, Beam: {}".format(8, tx_beam))

tx_beam = steervec(8,beamset[-3], 0)
print("Ntx: {}, Beam: {}".format(8, tx_beam))

#theta, gr = plotbeam(beamset[-2], 8)
#theta2, gr2 = plotbeam(beamset[2], 8)
#theta3, gr3 = plotbeam(beamset[0], 8)
#ax = plt.subplot(111, projection='polar')
##print(theta.shape, gr.shape)
#ax.plot(theta, gr, theta2, gr2, theta3, gr3)
#plt.show()
    

Beamset: [22.35, 33.8, 11.46, 28.07, 16.62, 45.26, 56.15, 50.42, 39.53, 67.61, 78.5, 73.34, 61.88, 89.95, 101.41, 95.68, 84.22, 112.3, 123.76, 118.03, 107.14, 135.22, 146.1, 140.37, 129.49, 157.56, 169.02, 163.29, 151.83, 179.91, 5.73, 174.18]
32
[0.39 0.59 0.2  0.49 0.29 0.79 0.98 0.88 0.69 1.18 1.37 1.28 1.08 1.57
 1.77 1.67 1.47 1.96 2.16 2.06 1.87 2.36 2.55 2.45 2.26 2.75 2.95 2.85
 2.65 3.14 0.1  3.04]
Ntx: 8, Beam: [ 0.35355339+0.j         -0.35350985+0.00554875j  0.35337922-0.01109613j
 -0.35316156+0.01664078j  0.3528569 -0.02218133j -0.35246532+0.02771642j
  0.35198692-0.03324468j -0.35142182+0.03876475j]
Ntx: 8, Beam: [ 0.35355339+0.00000000e+00j -0.35355339-1.40869652e-06j
  0.35355339+2.81739304e-06j -0.35355339-4.22608956e-06j
  0.35355339+5.63478608e-06j -0.35355339-7.04348259e-06j
  0.35355339+8.45217911e-06j -0.35355339-9.86087563e-06j]


In [6]:
#Choose the environment
em = EnvManager(device, 'combrf-v8', seed)
available_actions = em.num_actions_available()
random.seed(seed)
state_size = em.state_size()
print(em.env.ch_model)
print(em.env.sc_xyz)
#Select the strategy
strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay, 3500)

if PRIORITIZED_REPLAY:
    beta_strategy = EpsilonGreedyStrategy(PER_BETA, 1.0, eps_decay, 3500)

#Initialize the agent
agent = Agent(strategy, state_size, available_actions, seed, device)

#Instantiate MemoryBuffer
if not PRIORITIZED_REPLAY:
    memory = ReplayBuffer(available_actions, BUFFER_SIZE, BATCH_SIZE, seed, device)
else:
    memory = PrioritizedReplayBuffer(available_actions, BUFFER_SIZE, BATCH_SIZE, PER_ALPHA,seed, device)

policy_net = QNetwork(state_size, available_actions, seed).to(device)
target_net = QNetwork(state_size, available_actions, seed).to(device)
print(policy_net)

#Initialize target_net weights to policy_net weights
target_net.load_state_dict(policy_net.state_dict())
target_net.eval() #Set the target_net in eval mode

#Select the optimizer
optimizer = optim.Adam(params=policy_net.parameters(), lr=LR)

uma-nlos
[[-17.36481777  49.24038765  22.5       ]
 [-50.         -29.60405375  22.5       ]
 [ 17.10100717  93.96926208  22.5       ]
 [  4.35778714  98.26285905  22.5       ]]
QNetwork(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=5, out_features=400, bias=True)
    (1): Linear(in_features=400, out_features=400, bias=True)
    (2): Linear(in_features=400, out_features=64, bias=True)
  )
  (softmax): Softmax(dim=1)
  (output): Linear(in_features=64, out_features=8, bias=True)
)


In [7]:
print("tx_locs: {}".format(em.env.tx_locs))
print("Total locations: ", len(em.env.tx_locs))

tx_locs: [array([[93.96926208, 34.20201433, 25.        ]]), array([[93.96926208, 64.27876097, 25.        ]]), array([[93.96926208, 86.60254038, 25.        ]]), array([[93.96926208, 98.4807753 , 25.        ]]), array([[76.60444431, 34.20201433, 25.        ]]), array([[76.60444431, 64.27876097, 25.        ]]), array([[76.60444431, 86.60254038, 25.        ]]), array([[76.60444431, 98.4807753 , 25.        ]]), array([[50.        , 34.20201433, 25.        ]]), array([[50.        , 64.27876097, 25.        ]]), array([[50.        , 86.60254038, 25.        ]]), array([[50.       , 98.4807753, 25.       ]]), array([[17.36481777, 34.20201433, 25.        ]]), array([[17.36481777, 64.27876097, 25.        ]]), array([[17.36481777, 86.60254038, 25.        ]]), array([[17.36481777, 98.4807753 , 25.        ]])]
Total locations:  16


## Test with random untrained actions

In [8]:
obs = em.env.reset(np.exp(1j * 2 * np.pi * 0.6))
#print(len(em.env.beamwidth_vec))
#print(em.env.action_space.n)
print(obs)
ep_rwd=[]
while True:
    action = random.randrange(em.env.action_space.n)
    
    obs, rwd, done, _ = em.env.step(action)
    print(action)
    ep_rwd.append(rwd)
    if done:
        break
        
print("Episode score: {}".format(np.sum(ep_rwd)))

[[0.5        0.         0.5        0.34202014 1.        ]]
6
6
Episode score: -1.0


In [9]:
moving_rpts = [[np.array([rp_loc])] for rp_loc in em.env.sc_xyz]

#print("Initial: \n", moving_rpts)

for i in range(2):
    for ndx in range(len(moving_rpts)):
        if (ndx == 0) or (ndx == 1):
            last_loc = moving_rpts[ndx][-1]
            new_loc = np.array([[last_loc[0][0]+em.env.rx_stepsize, last_loc[0][1], last_loc[0][2]]]) 
            moving_rpts[ndx].append(new_loc)
        elif (ndx == 2) or (ndx == 3):
            last_loc = moving_rpts[ndx][-1]
            new_loc = np.array([[last_loc[0][0], last_loc[0][1]-em.env.rx_stepsize, last_loc[0][2]]]) 
            moving_rpts[ndx].append(new_loc)

print("After adding moving patterns: ", moving_rpts)



After adding moving patterns:  [[array([[-17.36481777,  49.24038765,  22.5       ]]), array([[82.63518223, 49.24038765, 22.5       ]]), array([[182.63518223,  49.24038765,  22.5       ]])], [array([[-50.        , -29.60405375,  22.5       ]]), array([[ 50.        , -29.60405375,  22.5       ]]), array([[150.        , -29.60405375,  22.5       ]])], [array([[17.10100717, 93.96926208, 22.5       ]]), array([[17.10100717, -6.03073792, 22.5       ]]), array([[  17.10100717, -106.03073792,   22.5       ]])], [array([[ 4.35778714, 98.26285905, 22.5       ]]), array([[ 4.35778714, -1.73714095, 22.5       ]]), array([[   4.35778714, -101.73714095,   22.5       ]])]]


In [10]:
eps_scpts = []

for i in range(episodes):
    #print(random.choice(moving_rpts[0]))
    sc_pts = np.array([random.choice(moving_rpts[ndx])[0] for ndx in range(len(moving_rpts))])
    eps_scpts.append(sc_pts)

#print(eps_scpts)

In [11]:
print(eps_scpts[7])

[[182.63518223  49.24038765  22.5       ]
 [-50.         -29.60405375  22.5       ]
 [ 17.10100717  -6.03073792  22.5       ]
 [  4.35778714  -1.73714095  22.5       ]]


## Train the DQN agent

In [12]:
ep_rewards = []
test_rewards = []
test_data_rates = []
test_eps_iters=[]
test_minexh_rates = []
test_maxexh_rates = []
false_positives =0
policy_net.train()
test_txbdir = 0

strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay, 4100)
outer = tqdm.notebook.tqdm(total=episodes, desc='training loop: ', position=0)
train_steps = 0
test_loc_ndx = 0
for episode in range(episodes):
    
    #if(eps <= 0.1):
    #    em.env.sc_xyz=np.array([])
    #    em.env.ch_model = 'uma-los'
    #if ((episode+1) == 3500):
    #    em.env.ch_model = 'uma-los'
    #    em.env.sc_xyz = np.array([])
        #train_steps = 0
        #strategy =EpsilonGreedyStrategy(1.0, eps_end, 0.997)#0.1
    #if ((episode+1) == 3900):
    #    train_steps=0
    #    strategy =EpsilonGreedyStrategy(1.0, eps_end, 0.9983, 3500)#0.1
    #    if PRIORITIZED_REPLAY:
    #        beta_strategy = EpsilonGreedyStrategy(PER_BETA, 1.0, eps_decay, 3500)
    #em.env.sc_xyz = eps_scpts[episode]
    #if (episode+1) == 4100:
    #    obs_tensor = torch.tensor(np.array([[-100,-100,21.5]]), device=device, dtype=torch.float32)
    #    print("TXloc: {}, obs_tensor: {}".format(np.array([[-100,-100,21.5]]), obs_tensor))
    #    for action in range(em.env.action_space.n):
    #        action_tensor = torch.tensor([action]).to(device)
    #        learnt_qval = policy_net(obs_tensor).gather(1,index=action_tensor)
    #        print("action: {}, learnt_qval: {}".format(action, learnt_qval))
        
    if ((episode+1) >= 4100):    
        #obs = em.test_reset(np.array([[100*np.cos(em.env.deg),100*np.sin(em.env.deg),25.0]]), test_txbdir, em.env.sc_xyz, ch_randvals[episode])
        obs = em.test_reset(em.env.tx_locs[test_loc_ndx], test_txbdir, em.env.sc_xyz, ch_randvals[episode])
        test_loc_ndx = (test_loc_ndx + 1) % len(em.env.tx_locs)
        #test_txbdir = (test_txbdir + 1) % em.env.obs_space.nvec[3]
    else:
        obs = em.reset(ch_randvals[episode])

    
    ep_loss = 0.0
    ep_rwd = 0.0
    timestep = 0
    tx_dirs = []
    rx_dirs = []
    data_rates =[]
    #agent.current_step +=1
    train_steps +=1
    eps = strategy.get_exploration_rate(train_steps)
    if PRIORITIZED_REPLAY:
        beta = beta_strategy.get_exploration_rate(train_steps)
    
    while True:
        tx_dirs.append(em.env.tx_bdir*(180/np.pi))
        rx_dirs.append(em.env.rx_bdir*(180/np.pi))
        data_rates.append(em.env.rate)

        action = agent.act(obs, policy_net, eps)
        next_obs, reward, done, _ = em.step(action)
        #agent.step(obs, action, reward, next_obs, done)
        ep_rwd += reward.item()
        
        min_exh_rate, max_exh_rate,min_action_ndx,max_action_ndx,_,_ = em.env.get_minmax_exhrate(ch_randvals[episode])
        '''
        if(np.all(em.env.tx_loc == np.array([[100,100,0]]))):
            print("ch_model: {}, sc_xyz: {}, min_exh_rate: {}, max_exh_rate: {}, min_action_ndx: {}, max_action_ndx: {}".format(em.env.ch_model, em.env.sc_xyz, min_exh_rate, max_exh_rate, min_action_ndx, max_action_ndx))
        '''   
        memory.add(obs, action, reward, next_obs, done)
        obs = next_obs
        
        if memory.can_provide_sample():
            
            if PRIORITIZED_REPLAY:
                experiences = memory.sample(beta)
                observations, actions, rewards, next_observations, dones, weights, batch_indices = experiences
            else:
                experiences = memory.sample()
                observations, actions, rewards, next_observations, dones = experiences
                weights, batch_indices = torch.tensor(np.ones_like(rewards.cpu().data.numpy())).to(device), None
            
            #print(states.shape, states.dtype)
            #print(actions.unsqueeze(-1).shape)
            current_q_values = policy_net(observations).gather(1,index=actions.unsqueeze(-1))#(1-ALPHA)*
            
            next_q_values = target_net(next_observations).detach().max(1)[0]
            target_q_values = ((next_q_values*GAMMA) + rewards)
            #print(policy_net(observations).type())
            #print(target_q_values.unsqueeze(1).type())
            #loss = F.mse_loss(current_q_values, target_q_values.unsqueeze(1).float())
            td_errors = ALPHA*(target_q_values.unsqueeze(1).float()-current_q_values)
            loss = F.mse_loss(torch.zeros(current_q_values.size()).to(device), ALPHA*(target_q_values.unsqueeze(1).float()-current_q_values),  reduction=None)
            #weighted_loss = torch.mean(weights*loss)
            #print("loss: ", loss)
            #loss = ALPHA*loss
            #ep_loss += weighted_loss.item()
            ep_loss += loss.item()
            
            optimizer.zero_grad()
            #weighted_loss.backward()
            loss.backward()
            optimizer.step()
            
            if PRIORITIZED_REPLAY:
                comp_errors = np.array([x[0] for x in td_errors.cpu().data.numpy()])
                new_priorities = np.abs(comp_errors) + 1e-6
                #print(new_priorities)
                memory.update_priorities(batch_indices.cpu().data.numpy(), new_priorities)
            
        timestep +=1
        if done:
            ep_rewards.append(ep_rwd)
            moving_avg_rwd = get_moving_average(100, ep_rewards)
            print('\rEpisode {},\tScore: {:.2f}, eps: {}, moving avg_rwd: {}, ep_loss: {}'.format(episode+1, ep_rwd, eps, moving_avg_rwd[-1], ep_loss), end="\r")
            #plot(episode_rewards, 100)
            
            #if(((episode+1) >= 6000) or (3000 <= (episode+1)<=3500)):
            if((episode+1) >= 4100):
                
                test_data_rates.append(data_rates[-1])
                test_eps_iters.append(timestep)
                test_minexh_rates.append(min_exh_rate)
                test_maxexh_rates.append(max_exh_rate)
                
                #if (timestep > 3):
                #    if(timestep >=8):
                #        false_positives +=1
                #if (min_exh_rate == max_exh_rate):
                print("\ntest_eps: {0}, eps_rwd: {1}".format(episode+1, ep_rwd))
                print("TX loc: {}".format(em.env.tx_loc))
                print("SC_xyz: {}".format(em.env.sc_xyz))
                print("TX dirs: ", tx_dirs)
                print("RX dirs: ", rx_dirs)
                print("data rates: {}".format(data_rates))
                print("min exh_rate: {}, min_action_ndx: {}, beamset val: {}".format(min_exh_rate, min_action_ndx, em.env.BeamSet[min_action_ndx]*180/np.pi))
                print("max exh_rate: {}, max_action_ndx: {}, beamset val: {}\n".format(max_exh_rate, max_action_ndx, em.env.BeamSet[max_action_ndx]*180/np.pi))
                
            timestep = 0
            break
            
        #print("timestep: {}".format(timestep))
    #if episode % UPDATE_EVERY == 0:
    #    target_net.load_state_dict(policy_net.state_dict())
    for local_param, target_param in zip(policy_net.parameters(), target_net.parameters()):
        target_param.data.copy_(TAU*local_param.data + (1.0-TAU)*target_param.data)
    
    
    '''    
    if ((episode == 0) or (episode+1) == 3100) or (((episode+1) > 3100) and ((train_steps) % (TEST_EVERY) == 0)):
        #test some episodes to check the performance
        policy_net.eval()
        test_ep_rwds = []
        for test_eps in range(test_episodes):
            obs = em.test_reset(test_loc, em.env.action_space.sample(), em.env.sc_xyz, ch_randvals[episode+test_eps+1])
            test_score=0
            tx_dirs = []
            rx_dirs = []
            data_rates =[]
            step = 0
            done = False
            while not done:
                action = policy_net(obs).argmax(dim=1).to(device)#agent.act(state_tensor, policy_net)
                #print("timestep: ",step, policy_net(obs),policy_net(obs).size(), action)
                tx_dirs.append(em.env.tx_bdir*(180/np.pi))
                rx_dirs.append(em.env.rx_bdir*(180/np.pi))
                data_rates.append(em.env.rate)
                next_obs, reward, done, _ = em.step(action)
                step+=1
                print("\ntstep: {0}, obs: {1}, action: {2}, next_obs: {3}, rwd: {4}, done: {5}".format(step, obs.cpu().data.numpy(), action.cpu().data.numpy(), next_obs.cpu().data.numpy(), reward.item(), done.item()))
                test_score+=reward.item()
                obs = next_obs
            
            print("test_eps: {0}, eps_rwd: {1}".format(test_eps+1, test_score))
            print("TX loc: {}".format(em.env.tx_loc))
            print("TX dirs: ", tx_dirs)
            print("RX dirs: ", rx_dirs)
            print("data rates: {}".format(data_rates))
            min_exh_rate, max_exh_rate,min_action_ndx,max_action_ndx,_,_ = em.env.get_minmax_exhrate(ch_randvals[episode+test_eps+1])
            print("min exh_rate: {}, min_action_ndx: {}".format(min_exh_rate, min_action_ndx))
            print("max exh_rate: {}, max_action_ndx: {}".format(max_exh_rate, max_action_ndx))
            
            test_ep_rwds.append(test_score)
            test_rewards.append(test_score)
            test_data_rates.append(data_rates[-1])
            test_minexh_rates.append(min_exh_rate)
            test_maxexh_rates.append(max_exh_rate)
            
        print("Average test_ep_score: {}\n".format(np.mean(test_ep_rwds)))
        policy_net.train()
        #agent.current_step=0
        train_steps = 0
        strategy =EpsilonGreedyStrategy(0.5, eps_end, 0.997)#0.1
    else:
        test_rewards.append(0)
        test_data_rates.append(0)
        test_minexh_rates.append(0)
        test_maxexh_rates.append(0)
    '''
    # update tqdm bar
    outer.update(1)
    
torch.save(policy_net.state_dict(), 'checkpoint.pth')

#timer.finish()
plot(ep_rewards, 200, test_rewards)
print("No. of false positives: {}".format(false_positives))

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [ ]:
fig=plt.figure()
min_ndx=0
max_ndx=500
plt.plot(test_eps_iters[min_ndx:max_ndx])
plt.show()
print(np.mean(test_eps_iters), len(test_eps_iters))

In [ ]:
print(em.env.ch_model)
print(em.env.sc_xyz)
print(np.mean(test_data_rates))

In [ ]:
fig = plt.figure(figsize=[9, 6])
min_ndx=24
max_ndx=48
plt.plot(np.arange(len(test_data_rates[min_ndx:max_ndx])), test_data_rates[min_ndx:max_ndx], 'b', np.arange(len(test_data_rates[min_ndx:max_ndx])), test_minexh_rates[min_ndx:max_ndx], 'r--', np.arange(len(test_data_rates[min_ndx:max_ndx])), test_maxexh_rates[min_ndx:max_ndx], 'g--')
#plt.plot(np.arange(len(test_data_rates[3080:3140])), test_data_rates[3080:3140], 'b', np.arange(len(test_data_rates[3080:3140])), test_minexh_rates[3080:3140], 'r--', np.arange(len(test_data_rates[3080:3140])), test_maxexh_rates[3080:3140],'g--')
#plt.plot(np.arange(len(test_data_rates[4650:4740])), test_data_rates[4650:4740], 'b', np.arange(len(test_data_rates[4650:4740])), test_minexh_rates[4650:4740], 'r--', np.arange(len(test_data_rates[4650:4740])), test_maxexh_rates[4650:4740],'g--')
#plt.plot(np.arange(len(test_data_rates[6230:])), test_data_rates[6230:], 'b', np.arange(len(test_data_rates[6230:])), test_minexh_rates[6230:], 'r--', np.arange(len(test_data_rates[6230:])), test_maxexh_rates[6230:],'g--')

plt.xticks(np.arange(0, max_ndx-min_ndx), [str(x) for x in np.arange(min_ndx, max_ndx)])
plt.legend(['learnt rate','min exhrate', 'max exhrate'])
plt.xlabel('Episode #')
plt.ylabel('data rate (bits/s)')
plt.title('Beam alignment with episode length:{}'.format(em.env.goal_steps))
plt.show()

In [ ]:
min_ndx=60
max_ndx=80
print(test_data_rates[min_ndx:max_ndx])
print(test_maxexh_rates[min_ndx:max_ndx])


In [ ]:
print(test_data_rates[69])
print([x*180/np.pi for x in em.env.BeamSet])
#print(em.env.BeamSet[9])

In [ ]:
print(em.env.action_space.n)
print(em.env.BeamSet)

In [ ]:
print(em.env.dqnobs_counter)
print(len(em.env.dqnobs_counter))
print(np.mean(em.env.dqnobs_counter))

In [ ]:
#import pickle

#memory.save('memory_checkpoint.pth')
plot(ep_rewards, 100, test_rewards)
print(ep_rewards[:100])
moving_avg_rwd = get_moving_average(100, ep_rewards[:100])
print(moving_avg_rwd)

## Test the DQN agent

In [ ]:
from Source.misc_fun.utils import var_plotbeam

print(tx_dirs)
print(rx_dirs)
for tx_ang, rx_ang in zip(tx_dirs, rx_dirs):
    tx_theta, tx_gr = var_plotbeam(tx_ang, em.env.N_tx)
    rx_theta, rx_gr = var_plotbeam(rx_ang, em.env.N_rx)
    ax1 = plt.subplot(122, projection='polar')
    ax1.plot(tx_theta, tx_gr)

    ax2 = plt.subplot(121, projection='polar')
    ax2.plot(rx_theta, rx_gr)

    plt.show()




## Display all exhaustive rate measurements from env

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from matplotlib.collections import PolyCollection
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

fig = plt.figure()
ax = fig.gca(projection='3d')
poly = PolyCollection(verts[:4], facecolors=['r', 'g', 'b', 'y'], alpha=0.6)
print(len(verts))
print(verts[0:2])
#print(np.array(tx_locs[:4]))
ax.add_collection3d(poly, zs=np.arange(1,5), zdir='y')

ax.set_xlabel('X (beam pairs)')
ax.set_ylabel('Y (tx_locs)')
ax.set_zlabel('Z (data rates)')
ax.set_xlim(0, 10)
ax.set_ylim(1, 5)
ax.set_zlim(10, 30)

plt.show()

In [ ]:
print(em.env.ch_model)
print(em.env.sc_xyz)
em.env.BeamSet[5][0]*180/np.pi
np.arctan(1100/200)*180/np.pi

In [ ]:
12/24, 14/24

from Source.misc_fun.utils import plotbeam

tx_theta, tx_gr = plotbeam(exh_txbeams[0]*(np.pi/180), em.env.N_tx)
rx_theta, rx_gr = plotbeam(exh_rxbeams[0]*(np.pi/180), em.env.N_rx)
ax1 = plt.subplot(122, projection='polar')
ax1.plot(tx_theta, tx_gr)

ax2 = plt.subplot(121, projection='polar')
ax2.plot(rx_theta, rx_gr)

plt.show()

